In [1]:
import time
import pandas as pd   

from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def scrape_yt(url, brand):
    data=[]

    with Chrome(executable_path=r'C:\Users\SYDNEY\AppData\Local\Programs\Python\Python37\Lib\chromedriver.exe') as driver:
        wait = WebDriverWait(driver,3)
        driver.get(url)
       
        for item in range(5): 
            wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
            time.sleep(3)
  
        # get the last comment
        lastEle = driver.find_element_by_xpath("(//*[@id='content-text'])[last()]")
        # scroll to the last comment currently loaded
        lastEle.location_once_scrolled_into_view
        # wait until the comments loading is done
        WebDriverWait(driver,5).until(EC.invisibility_of_element((By.CSS_SELECTOR,"div.active.style-scope.paper-spinner")))

        # load all comments
        while lastEle != driver.find_element_by_xpath("(//*[@id='content-text'])[last()]"):
            lastEle = driver.find_element_by_xpath("(//*[@id='content-text'])[last()]")
            driver.find_element_by_xpath("(//*[@id='content-text'])[last()]").location_once_scrolled_into_view
            time.sleep(2)
            WebDriverWait(driver,5).until(EC.invisibility_of_element((By.CSS_SELECTOR,"div.active.style-scope.paper-spinner")))

        # open all replies
        for reply in driver.find_elements_by_xpath("//*[@id='replies']//paper-button[@class='style-scope ytd-button-renderer'][contains(.,'View')]"):
            reply.location_once_scrolled_into_view
            driver.execute_script("arguments[0].click()",reply)
        time.sleep(2)
        WebDriverWait(driver, 5).until(
                EC.invisibility_of_element((By.CSS_SELECTOR, "div.active.style-scope.paper-spinner")))
        # print the total number of comments
        for comment in driver.find_elements_by_xpath("//*[@id='content-text']"):
            data.append(comment.text)
           
    df = pd.DataFrame(data, columns=['comment'])
    df['brand'] = brand
    df['url'] = url
    return df

In [3]:
url = 'https://www.youtube.com/watch?v=4osKpgjvdYo&ab_channel=ExplodingKittens'
name = 'ExplodingKittens'
df = scrape_yt(url, name) 
df.to_csv('kittens.csv', index=False)